In [6]:
#!pip install tensorflow

In [3]:
import tensorflow as tf 

In [5]:
print(tf.config.list_physical_devices('cpu'))

[]


In [7]:
import os
from pathlib import Path
import urllib.request as request
import numpy as np
import matplotlib.pyplot as plt

In [8]:
IMAGE_SIZE=(224,224)

In [9]:
model=tf.keras.applications.VGG16(
    #include_top=False->upto convolution layer
    include_top=True, weights="imagenet", input_tensor=None, input_shape=None,classes=1000
    
    
)

In [10]:
model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [11]:
model.save("VGG16_full_model.h5")

In [12]:
model.layers[1].padding

'same'

In [13]:
data_URL="https://download.pytorch.org/tutorial/hymenoptera_data.zip"

# Create a directory

In [14]:
def create_dirs(dir_path):
    os.makedirs(dir_path, exist_ok=True)
    print(f"{dir_path} directory created")

In [15]:
ROOT_DATA_DIR="data"
create_dirs(ROOT_DATA_DIR)

data directory created


# Downloading Data

In [16]:
data_zip_file="data.zip"
data_zip_file_path=os.path.join(ROOT_DATA_DIR,data_zip_file)


In [17]:
if not os.path.isfile(data_zip_file):
    print("Downloading dara...............................")
    filename, headers=request.urlretrieve(data_URL,data_zip_file_path)
    print(f"filename: {filename}  created with info \n{headers}")
else:
    print(f"file is already present")

filename: data\data.zip  created with info 
Content-Type: application/zip
Content-Length: 47286322
Connection: close
Date: Sat, 28 May 2022 14:19:56 GMT
Last-Modified: Wed, 15 Mar 2017 18:46:00 GMT
ETag: "5f8c32a6554f6acb4d649776e7735e48"
x-amz-version-id: null
Accept-Ranges: bytes
Server: AmazonS3
X-Cache: Miss from cloudfront
Via: 1.1 98b89ed6771805fd5b6d059738f20206.cloudfront.net (CloudFront)
X-Amz-Cf-Pop: TXL50-P3
X-Amz-Cf-Id: 2tF6V821dG3U3jIkzfmKdP1TiKLoBU0WPBMt1YRNpnpILOmmy2hRtg==




In [18]:
from zipfile import ZipFile

unzip_data_dirname="unzip_data_dir"
unzip_data_dir=os.path.join(ROOT_DATA_DIR,unzip_data_dirname)

if not os.path.exists(unzip_data_dir):
    os.makedirs(unzip_data_dir, exist_ok=True)
    with ZipFile(data_zip_file_path) as f:
        f.extractall(unzip_data_dir)
else:
    print(f"data already extracted")

In [28]:
main_data_dir=Path("./data/unzip_data_dir/hymenoptera_data/train")
main_data_dir

WindowsPath('data/unzip_data_dir/hymenoptera_data/train')

In [20]:
BATCH_SIZE=32
pixels=224
IMAGE_SIZE=(pixels, pixels)

In [22]:
datagen_kwargs=dict(rescale=1./255, validation_split=0.20)
dataflow_kwargs=dict(target_size=IMAGE_SIZE, batch_size=BATCH_SIZE)

In [23]:
datagen_kwargs

{'rescale': 0.00392156862745098, 'validation_split': 0.2}

In [24]:
dataflow_kwargs

{'target_size': (224, 224), 'batch_size': 32}

In [30]:
valid_datagen=tf.keras.preprocessing.image.ImageDataGenerator(**datagen_kwargs)
valid_generator=valid_datagen.flow_from_directory(
    main_data_dir, subset="validation", shuffle=False, **dataflow_kwargs)

do_data_augmentation=False
if do_data_augmentation:
    train_datagen=tf.keras.preprocessing.image.ImageDataGenerator(
        rotational_range=40,
        horizontal_flip=True,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2, zoom_range=0.2,
        **datagen_kwargs)
else:
    train_datagen=valid_datagen
train_generator=train_datagen.flow_from_directory(
        main_data_dir, subset="training", shuffle=True, **dataflow_kwargs
    )

Found 48 images belonging to 2 classes.
Found 196 images belonging to 2 classes.


In [31]:
RGB_IMAGE_SIZE=(pixels,pixels,3)
vgg=tf.keras.applications.vgg16.VGG16(
    input_shape=RGB_IMAGE_SIZE,
    weights="imagenet",
    include_top=False
    
)

In [32]:
vgg.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [33]:
for layer in vgg.layers:
    print(f"{layer.name:20s}: {layer.trainable}")

input_2             : True
block1_conv1        : True
block1_conv2        : True
block1_pool         : True
block2_conv1        : True
block2_conv2        : True
block2_pool         : True
block3_conv1        : True
block3_conv2        : True
block3_conv3        : True
block3_pool         : True
block4_conv1        : True
block4_conv2        : True
block4_conv3        : True
block4_pool         : True
block5_conv1        : True
block5_conv2        : True
block5_conv3        : True
block5_pool         : True


In [34]:
for layer in vgg.layers:
    layer.trainable=False

In [35]:
for layer in vgg.layers:
    print(f"{layer.name:20s}: {layer.trainable}")

input_2             : False
block1_conv1        : False
block1_conv2        : False
block1_pool         : False
block2_conv1        : False
block2_conv2        : False
block2_pool         : False
block3_conv1        : False
block3_conv2        : False
block3_conv3        : False
block3_pool         : False
block4_conv1        : False
block4_conv2        : False
block4_conv3        : False
block4_pool         : False
block5_conv1        : False
block5_conv2        : False
block5_conv3        : False
block5_pool         : False


In [37]:
del model

In [38]:
vgg.output

<KerasTensor: shape=(None, 7, 7, 512) dtype=float32 (created by layer 'block5_pool')>

In [41]:
classes=2
x=tf.keras.layers.Flatten()(vgg.output)
prediction=tf.keras.layers.Dense(classes, activation="softmax")(x)
model=tf.keras.models.Model(inputs=vgg.input, outputs=prediction)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [42]:
model.compile(
    optimizer=tf.keras.optimizers.SGD(lr=0.01, momentum=0.9),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=["accuracy"]
)

D:\PyTorch\PyTorch_Tutorial\env\lib\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [44]:
EPOCHS=30
history=model.fit(
    train_generator, epochs=EPOCHS, validation_data=valid_generator
)

Epoch 1/30
7/7 [==============================] - 99s 15s/step - loss: 35.8894 - accuracy: 0.5204 - val_loss: 25.4719 - val_accuracy: 0.5000
Epoch 2/30
7/7 [==============================] - 82s 13s/step - loss: 19.3430 - accuracy: 0.5459 - val_loss: 17.1663 - val_accuracy: 0.5000
Epoch 3/30
7/7 [==============================] - 81s 12s/step - loss: 6.9157 - accuracy: 0.6684 - val_loss: 1.9149 - val_accuracy: 0.7917
Epoch 4/30
7/7 [==============================] - 82s 11s/step - loss: 2.4717 - accuracy: 0.8316 - val_loss: 2.4899 - val_accuracy: 0.8125
Epoch 5/30
7/7 [==============================] - 66s 9s/step - loss: 0.8223 - accuracy: 0.9184 - val_loss: 2.0795 - val_accuracy: 0.7500
Epoch 6/30
7/7 [==============================] - 62s 10s/step - loss: 0.7344 - accuracy: 0.9184 - val_loss: 3.9304 - val_accuracy: 0.7917
Epoch 7/30
7/7 [==============================] - 61s 9s/step - loss: 1.1649 - accuracy: 0.8776 - val_loss: 4.8446 - val_accuracy: 0.8125
Epoch 8/30
7/7 [=========